## Titanic Survival Prediction 
<img src="Images/Titanic_Image.jpg" width="50%">


We'll build a classification tree using the [Titanic data](https://www.kaggle.com/c/titanic-gettingStarted/data) provided by Kaggle.

In [27]:
import pandas as pd
import numpy as np

In [28]:
# read in the data
df = pd.read_csv('./Datasets/Titanic_train.csv')
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


Let's choose our response variable and a few features, and review **how to handle categorical features**:

- **Survived:** This is our response variable, and is already encoded as 0=died and 1=survived.
- **Pclass:** These are the passenger class categories (1=first class, 2=second class, 3=third class). They are logically ordered, so we'll leave them as-is. (If the tree splits on this feature, the splits will occur at 1.5 or 2.5.)
- **Sex:** This is a binary category, so we should encode it as 0=female and 1=male. (If the tree splits on this feature, the split will occur at 0.5.)
- **Age:** This is a numeric feature, but we need to fill in the missing values.
- **Embarked:** This is the port they embarked from. There are three unordered categories, so we should create dummy variables and drop one level as usual.

### Data Pre-Processing

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


If you carefully observe the above summary of pandas, there are total 891 rows, Age shows only 714 (means missing), Embarked (2 missing) and Cabin missing a lot as well. Object data types are non-numeric so we have to find a way to encode them to numerical values. One such way is columnisation ie. factorize to the row values to column header.

#### Lets try to drop some of the columns which many not contribute much to our machine learning model such as Name, Ticket, Cabin etc.

In [30]:
cols = ['PassengerId','Name','Ticket','Cabin','Embarked','Fare','Parch','SibSp']
df = df.drop(cols,axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 4 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Age         714 non-null float64
dtypes: float64(1), int64(2), object(1)
memory usage: 27.9+ KB


#### If you observe carefully, Age has only 714 rows ( some missing values) . One easiest way is to drop the rows with missing values 

In [31]:
df_temp=df.dropna()
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 890
Data columns (total 4 columns):
Survived    714 non-null int64
Pclass      714 non-null int64
Sex         714 non-null object
Age         714 non-null float64
dtypes: float64(1), int64(2), object(1)
memory usage: 27.9+ KB


#### But this is loss of too much training data.   Pandas has a nice interpolate() function that will replace all the missing NaNs to interpolated values.

In [32]:
df['Age'] = df['Age'].interpolate()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 4 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Age         891 non-null float64
dtypes: float64(1), int64(2), object(1)
memory usage: 27.9+ KB


In [33]:
gender_mapping = {'male': 1,
                'female': 0}

df['Sex'] = df['Sex'].map(gender_mapping)
df

Survived  Pclass  Sex   Age
0           0       3    1  22.0
1           1       1    0  38.0
2           1       3    0  26.0
3           1       1    0  35.0
4           0       3    1  35.0
5           0       3    1  44.5
6           0       1    1  54.0
7           0       3    1   2.0
8           1       3    0  27.0
9           1       2    0  14.0
10          1       3    0   4.0
11          1       1    0  58.0
12          0       3    1  20.0
13          0       3    1  39.0
14          0       3    0  14.0
15          1       2    0  55.0
16          0       3    1   2.0
17          1       2    1  16.5
18          0       3    0  31.0
19          1       3    0  33.0
20          0       2    1  35.0
21          1       2    1  34.0
22          1       3    0  15.0
23          1       1    1  28.0
24          0       3    0   8.0
25          1       3    0  38.0
26          0       3    1  28.5
27          0       1    1  19.0
28          1       3    0  26.0
29          0       3    1  33.0
..        ...     ...  ...   ...
861         0       2    1  21.0
862         1       1    0  48.0
863         0       3    0  36.0
864         0       2    1  24.0
865         1       2    0  42.0
866         1       2    0  27.0
867         0       1    1  31.0
868         0       3    1  17.5
869         1       3    1   4.0
870         0       3    1  26.0
871         1       1    0  47.0
872         0       1    1  33.0
873         0       3    1  47.0
874         1       2    0  28.0
875         1       3    0  15.0
876         0       3    1  20.0
877         0       3    1  19.0
878         0       3    1  37.5
879         1       1    0  56.0
880         1       2    0  25.0
881         0       3    1  33.0
882         0       3    0  22.0
883         0       2    1  28.0
884         0       3    1  25.0
885         0       3    0  39.0
886         0       2    1  27.0
887         1       1    0  19.0
888         0       3    0  22.5
889         1       1    1  26.0
890         0       3    1  32.0

[891 rows x 4 columns]

# Time for Machien Learning 

Now we convert our dataframe from pandas to numpy and we assign input and output

In [34]:
X = df.values
y = df['Survived'].values

X = np.delete(X,1,axis=1)

#### Now that we are ready with X and y, lets split the dataset for 70% Training and  30% test set using scikit cross validation

In [35]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=0)

# Logistic Regression

In [36]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

In [37]:
import warnings
warnings.filterwarnings('ignore')

In [38]:
model=LogisticRegression(C=10)

In [39]:
model.fit(X_train,y_train)

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [40]:
model.score(X_test,y_test)

1.0

# Nive Bayes

In [41]:
from sklearn.naive_bayes import GaussianNB

In [42]:
model = GaussianNB()
model.fit(X_train, y_train)

GaussianNB()

In [43]:
expected = y_train
predicted = model.predict(X_train)

In [44]:
model.score(X_test,y_test)

1.0

#  Decision Tree

In [45]:
from sklearn.tree import DecisionTreeClassifier

In [46]:
model = DecisionTreeClassifier(max_depth=4,criterion='entropy')
model.fit(X_train, y_train)
print(model)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')


In [47]:
model.score(X_test,y_test)

1.0

# svm

In [48]:
from sklearn.svm import SVC
from sklearn import svm

In [49]:
model = svm.SVC(kernel='linear',gamma=1)
model.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=1, kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [50]:
model.score(X_test,y_test)

1.0

# knn

In [51]:
from sklearn.neighbors import KNeighborsClassifier

In [85]:
model = KNeighborsClassifier(n_neighbors=3)

In [86]:
model.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [87]:
model.score(X_test,y_test)

0.97388059701492535